In [9]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)

gzip: stdin

Exception: Unable to find py4j in /content/spark-3.5.2-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homesales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Referenced this for avg() function: https://www.w3schools.com/sql/sql_count_avg_sum.asp
# Referenced this for rounding function: https://www.w3schools.com/mysql/func_mysql_round.asp
# Tutor confirmed my approach to grouping by year is correct
spark.sql("""
SELECT year(date), round(avg(price),2)
FROM homesales
where bedrooms=4
group by year(date)
""").show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Tutor corrected my date reference
spark.sql("""
SELECT date_built, round(avg(price),2)
FROM homesales
Where bedrooms=3
and bathrooms=3
group by date_built
""").show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           292003.61|
+--------------------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# Tutor corrected my date reference
spark.sql("""
SELECT date_built, round(avg(price))
FROM homesales
where bedrooms=3
and bathrooms=3
and floors = 2
and sqft_living>=2000
group by date_built""").show()

+--------------------+
|round(avg(price), 0)|
+--------------------+
|            292538.0|
+--------------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
# Tutor helped order the 'group by' in the right location and put the 'having avg(price). Also helped format the Select statement
start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM homesales
group by view
having avg(price)>=350000
order by 1 DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------------------+
|round(avg((price / view)), 0)|
+-----------------------------+
|                      32606.0|
+-----------------------------+

--- 0.8756961822509766 seconds ---


In [ ]:
# 7. Cache the the temporary table homesales.
spark.sql("cache table homesales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('homesales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM homesales
group by view
having avg(price)>=350000
order by 1 DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------------------+
|round(avg((price / view)), 0)|
+-----------------------------+
|                      32606.0|
+-----------------------------+

--- 0.5350592136383057 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('delayed_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_homesales_p')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM p_homesales_p
group by view
having avg(price)>=350000
order by 1 DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------------------+
|round(avg((price / view)), 0)|
+-----------------------------+
|                      32606.0|
+-----------------------------+

--- 1.041222095489502 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homesales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("homesales")


False